# Импорт библиотек и константы

In [ ]:
!pip install datasets seqeval transformers --quiet

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import ast
import random
from collections import defaultdict, Counter
import re

In [ ]:
sns.set()

In [ ]:
import torch 
from torch import nn 
from torch.nn import CrossEntropyLoss
from torch.utils.data import Dataset, DataLoader 
from torch.optim import Adam
from transformers import get_linear_schedule_with_warmup, AutoTokenizer, AutoModelForPreTraining
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from datasets import Dataset, DatasetDict, load_metric
import torch.nn.functional as F

In [ ]:
# from google.colab import drive 

# drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/tatnashev/prozhito 

fatal: destination path 'prozhito' already exists and is not an empty directory.


In [ ]:
# import gdown 

# url = 'https://drive.google.com/file/d/1JH2lt4AZZjKD8z67GpOrpCT1IMQVtIfo/view?usp=sharing'
# gdown.download(url, 'ruberts.tar.gz', fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1JH2lt4AZZjKD8z67GpOrpCT1IMQVtIfo
To: /content/ruberts.tar.gz
100%|██████████| 1.98G/1.98G [00:15<00:00, 126MB/s]


'ruberts.tar.gz'

In [ ]:
# !tar -xf ruberts.tar.gz

"Константы"

In [ ]:
label_list = ['B-LOC', 'I-LOC', 'B-ORG', 'I-ORG', 'B-FAC', 'I-FAC', 'B-PER', 'I-PER', 'B-CHAR', 'I-CHAR', 'O']
# label_list = ['B-LOC', 'I-ORG', 'B-ORG', 'I-LOC', 'B-PER', 'O', 'I-PER']

bio2num = {
    'B-LOC': 0, 
    'I-LOC': 1, 
    'B-ORG': 2, 
    'I-ORG': 3, 
    'B-PER': 4, 
    'I-PER': 5, 
    'B-FAC': 6, 
    'I-FAC': 7, 
    'B-CHAR': 8, 
    'I-CHAR': 9, 
    'O': 10 
}

TOKENIZER_PATH = "DeepPavlov/rubert-base-cased"
MODEL_PATH = 'rubert_epoch_2'
# OUTPUT_DIR = '.'
res = {}
tokenizer = None 

In [ ]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [ ]:
set_random_seed(42)

# Загрузка данных 

In [ ]:
def preprocess_labels(labels): 
    nums = labels.split(', ')
    nums[0] = nums[0][1:] 
    nums[-1] = nums[-1][:-1]

    return list(map(int, nums))

In [ ]:
df_train_razmecheno = pd.read_csv('/content/prozhito/prozhito_data/df_train_prozhito.csv')
df_test_razmecheno = pd.read_csv('/content/prozhito/prozhito_data/df_test_prozhito.csv')
df_val_razmecheno = pd.read_csv('/content/prozhito/prozhito_data/df_val_prozhito.csv')

In [ ]:
df_razmecheno = pd.concat([df_train_razmecheno, df_val_razmecheno, df_test_razmecheno], ignore_index=True)

In [ ]:
df_razmecheno.head()

,Unnamed: 0,tokens,BIO_str,BIO_nums,BIO_list
0,0,У меня большая симпатия к Лукьянину — человек ...,O O O O O O O O O O O O O O O O O O O O O,"[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,2,"> Каким приговором , указом каким > Ты здесь ,...",O O O O O O O O O O O O O O O O O,"[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,3,"Подумал , что летом ребята куда затащили .",O O O O B-CHAR O O O,"[10, 10, 10, 10, 8, 10, 10, 10]","['O', 'O', 'O', 'O', 'B-CHAR', 'O', 'O', 'O']"
3,4,Нашел потрясающие материалы о В . М . Брадисе ...,O O O O B-PER I-PER I-PER I-PER I-PER O O B-CH...,"[10, 10, 10, 10, 4, 5, 5, 5, 5, 10, 10, 8, 8, ...","['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'I-PER'..."
4,5,[Без даты . ],O O O O,"[10, 10, 10, 10]","['O', 'O', 'O', 'O']"


In [ ]:
df_razmecheno = df_razmecheno.drop(columns=['Unnamed: 0']) 

In [ ]:
df_razmecheno.head()

,tokens,BIO_str,BIO_nums,BIO_list
0,У меня большая симпатия к Лукьянину — человек ...,O O O O O O O O O O O O O O O O O O O O O,"[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,"> Каким приговором , указом каким > Ты здесь ,...",O O O O O O O O O O O O O O O O O,"[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,"Подумал , что летом ребята куда затащили .",O O O O B-CHAR O O O,"[10, 10, 10, 10, 8, 10, 10, 10]","['O', 'O', 'O', 'O', 'B-CHAR', 'O', 'O', 'O']"
3,Нашел потрясающие материалы о В . М . Брадисе ...,O O O O B-PER I-PER I-PER I-PER I-PER O O B-CH...,"[10, 10, 10, 10, 4, 5, 5, 5, 5, 10, 10, 8, 8, ...","['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'I-PER'..."
4,[Без даты . ],O O O O,"[10, 10, 10, 10]","['O', 'O', 'O', 'O']"


In [ ]:
df_razmecheno['BIO_nums'] = df_razmecheno['BIO_nums'].apply(lambda x: preprocess_labels(x)) 

In [ ]:
df_train = pd.read_csv('/content/prozhito/data/ne5_train.csv')
df_test = pd.read_csv('/content/prozhito/data/ne5_test.csv')
df_val = pd.read_csv('/content/prozhito/data/ne5_val.csv')

In [ ]:
def preprocess_bio(bio): 
    bios = bio.split(', ') 
    bios[0] = bios[0][1:] 
    bios[-1] = bios[-1][:-1] 
    for i, x in enumerate(bios): 
        bios[i] = x[1:-1]

    return list(bios)

In [ ]:
df_train['BIO'] = df_train['BIO'].apply(lambda x: preprocess_bio(x))
df_test['BIO'] = df_test['BIO'].apply(lambda x: preprocess_bio(x))
df_val['BIO'] = df_val['BIO'].apply(lambda x: preprocess_bio(x))

In [ ]:
df_train.head()

,tokens,BIO
0,"['Я', 'голосовал', 'против', 'увольнения', 'Фе...","[O, O, O, O, B-PER, O, O, O, O, O, B-PER, O, O..."
1,"['Белоруссия', ',', 'Казахстан', ',', 'как', '...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
2,"['В', 'то', 'же', 'время', 'канцлер', 'подчерк...","[O, O, O, O, O, O, O, O, O, O, O, O, O]"
3,"['Спуск', 'корабля', 'с', 'орбиты', 'проходил'...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,"['Герой', 'Мураками', 'увлекается', 'западными...","[O, B-PER, O, O, O, O, O, O, O, O]"


In [ ]:
# vals = set()

# for bio in pd.concat([df_train, df_test, df_val], ignore_index=True)['BIO'].values: 
#     for x in bio: 
#         vals.add(x) 

# list(vals)

In [ ]:
def bio_to_num(bios): 
    ans = []
    for bio in bios: 
        ans.append(bio2num[bio])

    return ans

In [ ]:
df_train['BIO_nums'] = df_train['BIO'].apply(lambda x: bio_to_num(x))
df_test['BIO_nums'] = df_test['BIO'].apply(lambda x: bio_to_num(x))
df_val['BIO_nums'] = df_val['BIO'].apply(lambda x: bio_to_num(x))

df_train.head()

,tokens,BIO,BIO_nums
0,"['Я', 'голосовал', 'против', 'увольнения', 'Фе...","[O, O, O, O, B-PER, O, O, O, O, O, B-PER, O, O...","[10, 10, 10, 10, 4, 10, 10, 10, 10, 10, 4, 10,..."
1,"['Белоруссия', ',', 'Казахстан', ',', 'как', '...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1..."
2,"['В', 'то', 'же', 'время', 'канцлер', 'подчерк...","[O, O, O, O, O, O, O, O, O, O, O, O, O]","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1..."
3,"['Спуск', 'корабля', 'с', 'орбиты', 'проходил'...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1..."
4,"['Герой', 'Мураками', 'увлекается', 'западными...","[O, B-PER, O, O, O, O, O, O, O, O]","[10, 4, 10, 10, 10, 10, 10, 10, 10, 10]"


In [ ]:
# Считаем длину последовательности, 
# это понадобится дальше 

df_train['length'] = df_train['BIO_nums'].apply(lambda x: len(x))
df_test['length'] = df_test['BIO_nums'].apply(lambda x: len(x))
df_val['length'] = df_val['BIO_nums'].apply(lambda x: len(x))
df_razmecheno['length'] = df_razmecheno['BIO_nums'].apply(lambda x: len(x)) 

In [ ]:
data = DatasetDict({
    'train': Dataset.from_pandas(df_train[['tokens', 'BIO_nums', 'length']]),
    'test': Dataset.from_pandas(df_test[['tokens', 'BIO_nums', 'length']]),
    'val': Dataset.from_pandas(df_val[['tokens', 'BIO_nums', 'length']]), 
    'razmecheno_test': Dataset.from_pandas(df_razmecheno[['tokens', 'BIO_nums', 'length']])
})

data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'BIO_nums', 'length'],
        num_rows: 3160
    })
    test: Dataset({
        features: ['tokens', 'BIO_nums', 'length'],
        num_rows: 907
    })
    val: Dataset({
        features: ['tokens', 'BIO_nums', 'length'],
        num_rows: 448
    })
    razmecheno_test: Dataset({
        features: ['tokens', 'BIO_nums', 'length'],
        num_rows: 1678
    })
})

In [ ]:
# data = data.remove_columns('__index_level_0__')
# data

# Подготовка данных 

In [ ]:
def tokenizer_and_align(texts): 
    tokenized_input = tokenizer(texts['tokens'], truncation=True)
    print(tokenized_input.keys())

    labels = []
    for i, label in enumerate(texts['BIO_nums']): 
        word_ids = tokenized_input.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids: 
            if word_idx is None: 
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                if word_idx > len(label) - 1: 
                    label_ids.append(len(label_list) - 1)
                else: 
                    label_ids.append(label[word_idx])
            else: 
                if word_idx > len(label) - 1: 
                    label_ids.append(len(label_list) - 1)
                else:
                    label_ids.append(label[previous_word_idx])

            previous_word_idx = word_idx
            

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]
        labels.append(label_ids)

    tokenized_input['labels'] = labels 
    return tokenized_input 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH, model_max_length=512)

In [ ]:
tokenized_datasets = data.map(tokenizer_and_align, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns('tokens')

  0%|          | 0/4 [00:00<?, ?ba/s]

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


  0%|          | 0/1 [00:00<?, ?ba/s]

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


  0%|          | 0/1 [00:00<?, ?ba/s]

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


  0%|          | 0/2 [00:00<?, ?ba/s]

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


## Сортировка по длине батчей

In [ ]:
tokenized_datasets = tokenized_datasets.sort('length')

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns('length')
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['BIO_nums', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3160
    })
    test: Dataset({
        features: ['BIO_nums', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 907
    })
    val: Dataset({
        features: ['BIO_nums', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 448
    })
    razmecheno_test: Dataset({
        features: ['BIO_nums', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1678
    })
})

# Обучение

In [ ]:
def get_key_to_results(model_path, epochs, lr, weight_decay): 
    model = model_path.split('/')[-1]
    k = f'{model}, epochs = {epochs}, lr = {lr}, weight_decay = {weight_decay}'
    return k 

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(MODEL_PATH, num_labels=len(bio2num))
model.config.id2label = dict(enumerate(list(bio2num.keys())))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

Some weights of the model checkpoint at rubert_epoch_1 were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at rubert_epoch_1 and are newly initialized: 

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
metric = load_metric("seqeval")

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
# BIO_weights = [1, 1, 1, 1, 1, 1, 1]

In [ ]:
# normedWeights = torch.FloatTensor(BIO_weights)

# class MyTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False):
        
#         if "labels" in inputs:
#           labels = inputs.pop("labels")
        
#         outputs = model(**inputs)
#         logits = outputs.logits
#         # print('logits', logits.shape)
#         # print('logits', logits.permute(0, 2, 1).shape)
#         logits = logits.permute(0, 2, 1)
#         # print('labels', labels.shape)
#         loss_function = CrossEntropyLoss(weight = normedWeights)

#         if self.args.past_index >= 0:
#             self._past = outputs[self.args.past_index]

#         if labels is not None:
#             loss = loss_function(logits, labels)
#         else:
#             # We don't use .loss here since the model may return tuples instead of ModelOutput.
#             loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]

#         return (loss, outputs) if return_outputs else loss

In [ ]:
def predict(model, trainer, test_column): 
    predictions, labels, _ = trainer.predict(tokenized_datasets[test_column])
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    
    # to dataframe 
    domains_ans = []
    f1s = []
    numbers = []
    precs = []
    recalls = []

    for key in sorted(results.keys()):
        if 'overall' not in key:
            domains_ans.append(key)
            f1s.append(round(results[key]['f1'], 4))
            numbers.append(round(results[key]['number'], 4))
            precs.append(round(results[key]['precision'], 4))
            recalls.append(round(results[key]['recall'], 4))
        else:
            if key == 'overall_f1': 
                f1s.append(round(results[key], 4))
            elif key == 'overall_precision': 
                precs.append(round(results[key], 4)) 
            elif key == 'overall_recall': 
                recalls.append(round(results[key], 4))

    domains_ans.append('Overall')
    numbers.append(sum(numbers))

    # print(len(domains_ans), len(f1s), len(numbers), len(precs), len(recalls))

    to_add = pd.DataFrame({'tag': domains_ans,
                'f1': f1s,
                'precision': precs,
                'recall': recalls,
                'number of occurence': numbers})
    
    return to_add 


In [ ]:
def train(model, config): 
    model = AutoModelForTokenClassification.from_pretrained(MODEL_PATH, num_labels=len(label_list))
    model.config.id2label = dict(enumerate(label_list))
    model.config.label2id = {v: k for k, v in model.config.id2label.items()}

    epochs = config['epochs']
    lr = config['lr']
    weight_decay = config['weight_decay']
    batch_size = config['bs']

    args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        evaluation_strategy = "epoch",
        learning_rate=lr,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        weight_decay=weight_decay,
        report_to='none',
        save_strategy='no', 
        # load_best_model_at_end=True, 
        logging_dir=OUTPUT_DIR,
        logging_strategy='epoch'
    )

    trainer = Trainer(
        model,
        args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['val'],
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    for param in model.parameters():
        param.requires_grad = True

    import logging
    from transformers.trainer import logger as noisy_logger
    noisy_logger.setLevel(logging.WARNING)

    trainer.train() 

    return trainer 


In [ ]:
configs = [
    {
        'epochs': 7, 
        'lr': 2e-5, 
        'weight_decay': 1e-4, 
        'bs': 8
    }
    # {
    #     'epochs': 10, 
    #     'lr': 2e-5, 
    #     'weight_decay': 1e-4, 
    #     'bs': 8

    # }, 
    # {
    #     'epochs': 10, 
    #     'lr': 1e-5, 
    #     'weight_decay': 1e-5, 
    #     'bs': 8
    # }   
]

In [ ]:
histories = [] 

In [ ]:
for config in configs: 
    model = None 
    trainer = train(model, config)
    to_add = predict(model, trainer, 'test') 
    to_add_razmecheno = predict(model, trainer, 'razmecheno_test') 
    new_key = get_key_to_results(MODEL_PATH, config['epochs'], config['lr'], config['weight_decay'])

    res[new_key] = to_add 
    res['razmecheno'] = to_add_razmecheno 
    histories.append(trainer.state.log_history)

    del model 
    # del trainer 
    torch.cuda.empty_cache()

Some weights of the model checkpoint at rubert_epoch_1 were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at rubert_epoch_1 and are newly initialized: 

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.133700,0.092344,0.173684,0.058615,0.087649,0.975490
2,0.080300,0.068674,0.400000,0.174067,0.242574,0.977408
3,0.060900,0.063092,0.359102,0.255773,0.298755,0.978436
4,0.050500,0.056887,0.513298,0.342806,0.411076,0.981937
5,0.041800,0.054598,0.505669,0.396092,0.444223,0.982493
6,0.035300,0.052088,0.525271,0.516874,0.521038,0.984105
7,0.030500,0.053452,0.516522,0.527531,0.521968,0.984605


Configuration saved in ./checkpoint-500/config.json
Model weights saved in ./checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./checkpoint-500/tokenizer_config.json
Special tokens file saved in ./checkpoint-500/special_tokens_map.json
Configuration saved in ./checkpoint-1000/config.json
Model weights saved in ./checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./checkpoint-1000/special_tokens_map.json
Configuration saved in ./checkpoint-1500/config.json
Model weights saved in ./checkpoint-1500/pytorch_model.bin
tokenizer config file saved in ./checkpoint-1500/tokenizer_config.json
Special tokens file saved in ./checkpoint-1500/special_tokens_map.json
Configuration saved in ./checkpoint-2000/config.json
Model weights saved in ./checkpoint-2000/pytorch_model.bin
tokenizer config file saved in ./checkpoint-2000/tokenizer_config.json
Special tokens file saved in ./checkpoint-2000/special_toke

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# from collections import defaultdict

In [ ]:
# train_history = defaultdict(list)
# val_history = defaultdict(list)

# for i, states in enumerate(trainer.state.log_history[:-1]): 
#     tmp = {}
#     # print(states)

#     if i % 2: # even - train, odd - val  
#         val_history['acc'].append(states['eval_accuracy'])  
#         val_history['f1'].append(states['eval_f1']) 
#         val_history['precision'].append(states['eval_precision']) 
#         val_history['recall'].append(states['eval_recall']) 
#         val_history['loss'].append(states['eval_loss'])  
#     else: 
#         # loss = 0 
#         # if states['epoch'] >= 5.0:
#         #     loss = states['train_loss']
#         # else: 
#         #     loss = states['loss'] 

#         train_history['loss'].append(states['loss']) 

### Рисуем графики с метриками полученной модельки

In [ ]:
# f, axs = plt.subplots(2, 2, figsize=(25, 11)) 
# xaxis = np.arange(1, config['epochs'] + 1)

# title = list(res.keys())[0]

# f.suptitle(list(res.keys())[0], fontsize=14)

# # first plot - val and train losses
# axs[0, 0].plot(xaxis, val_history['loss'], label='val') 
# axs[0, 0].plot(xaxis, train_history['loss'], label='train') 
# axs[0, 0].legend() 
# axs[0, 0].set(title='Train an Val losses', xlabel='epochs') 

# # second plot - precision and recall per epoch 
# axs[0, 1].plot(xaxis, val_history['precision'], label='precision', color='darkgreen') 
# axs[0, 1].plot(xaxis, val_history['recall'], label='recall', color='darkmagenta')
# axs[0, 1].legend() 
# axs[0, 1].set(title='Precision and Recall on validation data', xlabel='epochs')

# # third plot - f1 per epoch 
# axs[1, 0].plot(xaxis, val_history['f1'], color='firebrick') 
# axs[1, 0].set(title='F1 score', xlabel='epochs')


# f.delaxes(axs[1, 1])
# f.show()

In [ ]:
# f.savefig('rubert_tiny.png')

### Результаты

In [ ]:
tmp = pd.concat(res) 
tmp

tag      f1  \
rubert_epoch_1, epochs = 7, lr = 2e-05, weight_... 0      FAC  0.6409   
                                                   1      LOC  0.4407   
                                                   2      ORG  0.4749   
                                                   3  Overall  0.5304   
razmecheno                                         0     CHAR  0.0000   
                                                   1      FAC  0.0205   
                                                   2      LOC  0.0000   
                                                   3      ORG  0.0000   
                                                   4      PER  0.0000   
                                                   5  Overall  0.0128   

                                                      precision  recall  \
rubert_epoch_1, epochs = 7, lr = 2e-05, weight_... 0     0.6571  0.6256   
                                                   1     0.4505  0.4313   
                                                   2     0.4852  0.4650   
                                                   3     0.5426  0.5187   
razmecheno                                         0     0.0000  0.0000   
                                                   1     0.0629  0.0122   
                                                   2     0.0000  0.0000   
                                                   3     0.0000  0.0000   
                                                   4     0.0000  0.0000   
                                                   5     0.0556  0.0073   

                                                      number of occurence  
rubert_epoch_1, epochs = 7, lr = 2e-05, weight_... 0                  438  
                                                   1                  211  
                                                   2                  529  
                                                   3                 1178  
razmecheno                                         0                  344  
                                                   1                  737  
                                                   2                  129  
                                                   3                   16  
                                                   4                   13  
                                                   5                 1239